In [18]:
# fixed_income_quiz.py
# Quiz-ready toolkit:
# - NSS curve: zcy(T), P(0,T), f(0,T)
# - Vasicek: ZC price, ZC option (via generic Gaussian-affine bond option)
# - CIR: ZC price (option stub to plug your course formula)
# - Hull-White (1F): uses NSS curve P(0,T) as market curve, prices bond options, coupon bond options (Jamshidian), caps/floors
#
# Requirements: numpy, scipy, matplotlib (plot cell optional)
# pip install numpy scipy matplotlib

from dataclasses import dataclass
import numpy as np
from math import exp, log, sqrt
from scipy.stats import norm
from scipy.optimize import brentq

# =========================
# 1) NSS (market curve input)
# =========================

@dataclass(frozen=True)
class NSSParams:
    a: float
    b: float
    c: float
    d: float
    tau: float
    theta: float


def zcy_nss(T, p: NSSParams, eps=1e-10):
    """
    Zero-coupon yield zcy_NSS(T) per your notes.
    T in years.
    """
    if p.tau <= 0 or p.theta <= 0:
        raise ValueError("tau and theta must be > 0")

    T = np.asarray(T, dtype=float)
    Ts = np.maximum(T, eps)

    x1 = Ts / p.tau
    x2 = Ts / p.theta

    term1 = (1.0 - np.exp(-x1)) / x1
    term2 = term1 - np.exp(-x1)
    term3 = (1.0 - np.exp(-x2)) / x2 - np.exp(-x2)

    y = p.a + p.b * term1 + p.c * term2 + p.d * term3
    return float(y) if np.isscalar(T) else y


def fwd_nss(T, p: NSSParams, eps=1e-10):
    """
    Instantaneous forward f_NSS(T) as in your notes (Eq. 2.6).
    """
    if p.tau <= 0 or p.theta <= 0:
        raise ValueError("tau and theta must be > 0")

    T = np.asarray(T, dtype=float)
    Ts = np.maximum(T, eps)

    part1 = (np.exp(-Ts / p.tau) / p.tau) * (p.b * p.tau + p.c * Ts)
    part2 = (np.exp(-Ts / p.theta) / p.theta) * (p.d * Ts)

    f = p.a + part1 + part2
    return float(f) if np.isscalar(T) else f


def P0T_from_nss(T, p: NSSParams, comp="cont"):
    """
    Discount factor P(0,T) from NSS zero yields.
    comp='cont' => P(0,T)=exp(-y(T)*T)
    """
    y = zcy_nss(T, p)
    T = np.asarray(T, dtype=float)
    if comp != "cont":
        raise ValueError("Only continuous compounding implemented (comp='cont').")
    P = np.exp(-np.asarray(y) * T)
    return float(P) if np.isscalar(T) else P


# =========================
# 2) Generic Gaussian-affine bond option (works for Vasicek & Hull-White with given curve)
# =========================

def sigma_p_hw(a_hw, sigma_hw, S, T):
    """
    Bond price volatility for option expiring at S on ZC maturing at T, under 1F Gaussian (HW/Vasicek form).
    Standard formula:
    sigma_p = sigma * (1 - exp(-a (T-S))) / a * sqrt((1 - exp(-2 a S)) / (2 a))
    """
    if S <= 0 or T <= S:
        raise ValueError("Need 0 < S < T.")
    if a_hw <= 0 or sigma_hw < 0:
        raise ValueError("Need a_hw > 0 and sigma_hw >= 0.")
    factor = (1.0 - exp(-a_hw * (T - S))) / a_hw
    vol = sigma_hw * factor * sqrt((1.0 - exp(-2.0 * a_hw * S)) / (2.0 * a_hw))
    return vol


def zc_option_gaussian(P0S, P0T, K, a, sigma, S, T, option_type="call"):
    """
    European option at t=0 on ZC bond P(S,T), expiry S, maturity T.
    Uses Black-like closed form valid for 1F Gaussian short-rate models (HW/Vasicek),
    when you feed the correct P0S and P0T (from market curve or model).
    """
    vol = sigma_p_hw(a, sigma, S, T)
    if vol <= 0:
        # deterministic case
        forward = P0T / P0S
        intrinsic = max(P0T - K * P0S, 0.0) if option_type == "call" else max(K * P0S - P0T, 0.0)
        return intrinsic

    h = (log(P0T / (K * P0S)) / vol) + 0.5 * vol
    if option_type == "call":
        price = P0T * norm.cdf(h) - K * P0S * norm.cdf(h - vol)
    elif option_type == "put":
        price = K * P0S * norm.cdf(-h + vol) - P0T * norm.cdf(-h)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    return price


# =========================
# 3) Vasicek model (analytic ZC)
# dr = kappa*(mu - r) dt + sigma dW  (risk-neutral form in many courses)
# =========================

@dataclass(frozen=True)
class VasicekParams:
    kappa: float
    mu: float
    sigma: float
    r0: float


def vasicek_B(kappa, T):
    return (1.0 - exp(-kappa * T)) / kappa


def vasicek_A(kappa, mu, sigma, T):
    B = vasicek_B(kappa, T)
    term1 = (mu - (sigma**2) / (2.0 * kappa**2)) * (B - T)
    term2 = (sigma**2) * (B**2) / (4.0 * kappa)
    return exp(term1 - term2)


def P0T_vasicek(T, vp: VasicekParams):
    if T < 0:
        raise ValueError("T must be >= 0")
    if T == 0:
        return 1.0
    if vp.kappa <= 0 or vp.sigma < 0:
        raise ValueError("Need kappa > 0 and sigma >= 0.")
    A = vasicek_A(vp.kappa, vp.mu, vp.sigma, T)
    B = vasicek_B(vp.kappa, T)
    return A * exp(-B * vp.r0)


def zc_option_vasicek(K, S, T, vp: VasicekParams, option_type="call"):
    P0S = P0T_vasicek(S, vp)
    P0T = P0T_vasicek(T, vp)
    # Same gaussian option engine (with a=kappa, sigma=sigma)
    return zc_option_gaussian(P0S, P0T, K, vp.kappa, vp.sigma, S, T, option_type=option_type)


# =========================
# 4) CIR model (analytic ZC)
# dr = kappa*(mu - r) dt + sigma*sqrt(r) dW
# =========================

@dataclass(frozen=True)
class CIRParams:
    kappa: float
    mu: float
    sigma: float
    r0: float


def cir_gamma(kappa, sigma):
    return sqrt(kappa*kappa + 2.0*sigma*sigma)


def cir_B(kappa, mu, sigma, T):
    g = cir_gamma(kappa, sigma)
    num = 2.0 * (exp(g*T) - 1.0)
    den = (g + kappa) * (exp(g*T) - 1.0) + 2.0 * g
    return num / den


def cir_A(kappa, mu, sigma, T):
    g = cir_gamma(kappa, sigma)
    den = (g + kappa) * (exp(g*T) - 1.0) + 2.0 * g
    num = 2.0 * g * exp((kappa + g) * T / 2.0)
    power = (2.0 * kappa * mu) / (sigma * sigma)
    return (num / den) ** power


def P0T_cir(T, cp: CIRParams):
    if T < 0:
        raise ValueError("T must be >= 0")
    if T == 0:
        return 1.0
    if cp.kappa <= 0 or cp.sigma <= 0:
        raise ValueError("Need kappa > 0 and sigma > 0 for CIR.")
    A = cir_A(cp.kappa, cp.mu, cp.sigma, T)
    B = cir_B(cp.kappa, cp.mu, cp.sigma, T)
    return A * exp(-B * cp.r0)


def zc_option_cir_stub(*args, **kwargs):
    """
    PLACEHOLDER:
    CIR ZC option has an analytic expression (often via noncentral chi-square terms),
    but notations vary by course. Plug your course formula here.
    """
    raise NotImplementedError(
        "CIR ZC option formula depends on your course notation/measures. "
        "Paste the exact formula from your notes here and this function is done."
    )


# =========================
# 5) Hull-White (1F) using NSS as market curve
# =========================

@dataclass(frozen=True)
class HullWhiteParams:
    a: float
    sigma: float


def P0T_hw_from_nss(T, nss: NSSParams):
    # HW fits the initial curve: use market P(0,T) from NSS
    return P0T_from_nss(T, nss, comp="cont")


def zc_option_hw_from_nss(K, S, T, nss: NSSParams, hw: HullWhiteParams, option_type="call"):
    P0S = P0T_hw_from_nss(S, nss)
    P0T = P0T_hw_from_nss(T, nss)
    return zc_option_gaussian(P0S, P0T, K, hw.a, hw.sigma, S, T, option_type=option_type)


# =========================
# 6) Coupon bond pricing + option (Jamshidian) for Gaussian models (HW/Vasicek engine)
# =========================

def coupon_bond_price_from_curve(cashflows, times, P0T_func):
    """
    Price at t=0 using a discount curve: sum_i CF_i * P(0, t_i)
    """
    cashflows = np.asarray(cashflows, dtype=float)
    times = np.asarray(times, dtype=float)
    if cashflows.shape != times.shape:
        raise ValueError("cashflows and times must have same shape")
    P = np.array([P0T_func(t) for t in times], dtype=float)
    return float(np.sum(cashflows * P))


def _find_rstar_for_jamshidian(K, cashflows, times, A_list, B_list):
    """
    Finds r* such that sum_i CF_i * A_i * exp(-B_i r*) = K
    (bond price at expiry S as function of short rate r_S, in Gaussian affine models).
    """
    def f(r):
        return np.sum(cashflows * A_list * np.exp(-B_list * r)) - K

    # Robust bracket (rates in decimals). Adjust if needed.
    lo, hi = -0.10, 0.50
    flo, fhi = f(lo), f(hi)
    # Expand if needed
    for _ in range(10):
        if flo * fhi < 0:
            break
        lo -= 0.10
        hi += 0.10
        flo, fhi = f(lo), f(hi)
    if flo * fhi >= 0:
        raise RuntimeError("Could not bracket root for r*. Check inputs / bracket range.")
    return brentq(f, lo, hi)


def coupon_bond_option_hw_from_nss_jamshidian(K, S, cashflows, times, nss: NSSParams, hw: HullWhiteParams, option_type="call"):
    """
    European option at t=0 with expiry S on a coupon bond paying CF_i at times t_i (t_i > S).
    Jamshidian decomposition for 1F Gaussian models.

    Steps:
    - Express P(S, t_i) = A_i * exp(-B_i * r_S)
      For Gaussian models, B_i = B(S, t_i) = (1-exp(-a(t_i-S)))/a
      and A_i chosen to fit initial curve: A_i = P(0,t_i) / (P(0,S) * exp(-B_i * f(0,S)?))
      In practice for HW with market curve, use standard ZC option engine:
      find r* using affine form at S, then sum ZC options with strikes K_i = P(S,t_i; r*).
    Here we implement using the known fact:
      Coupon bond option = sum_i CF_i * option on ZC with strike K_i
    where K_i is ZC price at S when r_S=r*.
    """
    cashflows = np.asarray(cashflows, dtype=float)
    times = np.asarray(times, dtype=float)
    if np.any(times <= S):
        raise ValueError("All coupon times must be > S for an option expiring at S.")

    # Compute B_i
    a = hw.a
    B_list = (1.0 - np.exp(-a * (times - S))) / a  # B(S, t_i)

    # For HW fitted to curve, ZC bond price at time S can be written:
    # P(S,t) = P(0,t)/P(0,S) * exp( -B(S,t)*x_S - 0.5*V(S,t) )
    # where x_S is Gaussian state. For Jamshidian root we need monotone in state.
    #
    # Instead of rebuilding A_i exactly, we can use the strike construction:
    # Find x* such that sum CF_i * P(S,t_i; x*) = K.
    # Because P(S,t) is exponential-affine in x, this is equivalent to root in x.
    #
    # We implement a root in x using the ratio form with market curve; then K_i are obtained
    # and each term priced as ZC option with strike K_i.

    P0S = P0T_from_nss(S, nss)

    # Precompute P0Ti and the variance term V(S,t_i) for the affine representation in x
    P0Ti = np.array([P0T_from_nss(t, nss) for t in times], dtype=float)

    # Variance term for log bond price under HW:
    # V(S,t) = sigma^2 * (1 - exp(-a(t-S)))^2 * (1 - exp(-2aS)) / (2a^3)
    def V_hw(S_, t_):
        return (hw.sigma**2) * ((1.0 - exp(-a*(t_ - S_)))**2) * (1.0 - exp(-2.0*a*S_)) / (2.0 * a**3)

    V_list = np.array([V_hw(S, t) for t in times], dtype=float)

    # Bond price at S as function of state x: P(S,t) = (P0t/P0S) * exp( -B*x - 0.5*V )
    def P_S_t_of_x(x):
        return (P0Ti / P0S) * np.exp(-B_list * x - 0.5 * V_list)

    def bond_S_of_x(x):
        return float(np.sum(cashflows * P_S_t_of_x(x)))

    # Root find x* such that bond_S_of_x(x*) = K
    def g(x):
        return bond_S_of_x(x) - K

    lo, hi = -5.0, 5.0
    glo, ghi = g(lo), g(hi)
    for _ in range(12):
        if glo * ghi < 0:
            break
        lo -= 2.0
        hi += 2.0
        glo, ghi = g(lo), g(hi)
    if glo * ghi >= 0:
        raise RuntimeError("Could not bracket root for x*. Check K and cashflows.")

    x_star = brentq(g, lo, hi)

    # Strikes K_i = P(S, t_i) at x_star
    K_i = P_S_t_of_x(x_star)

    # Option = sum CF_i * option_on_ZC_i with strike K_i
    # Each ZC option is priced with gaussian engine using P0S and P0Ti
    total = 0.0
    for P0T_i, Ki, ti in zip(P0Ti, K_i, times):
        total += float(cashflows[np.where(times == ti)][0]) * zc_option_gaussian(P0S, P0T_i, Ki, hw.a, hw.sigma, S, ti, option_type=option_type)

    return total


# =========================
# 7) Caps & Floors (decomposition into ZC options)
# =========================

def cap_floor_from_zc_options(
    notional,
    K,
    schedule,   # list of tuples (T_i, T_{i+1}, delta)
    nss: NSSParams,
    hw: HullWhiteParams,
    kind="cap"
):
    """
    Caps/Floors under HW using decomposition into options on ZC.
    
    Formulas (from course notes, Eq. 1.40 and 1.41):
    - Cap(M,K) = Σ M' · Put(P(t_{j-1}, t_j), K')
    - Floor(M,K) = Σ M' · Call(P(t_{j-1}, t_j), K')
    
    where M' = M(1 + Kτ) and K' = 1/(1 + Kτ)
    
    Parameters
    ----------
    notional : float
        Notional amount M
    K : float
        Strike rate (e.g., 0.03 for 3%)
    schedule : list of tuples (T_i, T_{i+1}, delta)
        Each tuple represents a caplet/floorlet period
    nss : NSSParams
        NSS curve parameters
    hw : HullWhiteParams
        Hull-White model parameters
    kind : str
        'cap' or 'floor'
    
    Returns
    -------
    float
        Price of the cap or floor
    """
    if kind not in ("cap", "floor"):
        raise ValueError("kind must be 'cap' or 'floor'")
    
    total = 0.0
    
    for (Ti, Tip1, delta) in schedule:
        # Modified notional and strike per course formulas
        M_prime = notional * (1.0 + K * delta)
        K_prime = 1.0 / (1.0 + K * delta)
        
        # For CAP: use PUT on ZC bond P(Ti, Ti+1)
        # For FLOOR: use CALL on ZC bond P(Ti, Ti+1)
        option_type = "put" if kind == "cap" else "call"
        
        # Price the option on the zero-coupon bond P(Ti, Ti+1)
        # Expiry = Ti, Maturity = Ti+1, Strike = K_prime
        option_price = zc_option_hw_from_nss(
            K=K_prime,
            S=Ti,
            T=Tip1,
            nss=nss,
            hw=hw,
            option_type=option_type
        )
        
        # Add contribution: M' × Option
        total += M_prime * option_price
    
    return total


# =========================
# 8) Quick examples (you can delete before quiz)
# =========================

if __name__ == "__main__":
    # --- Example NSS parameters (replace with quiz inputs)
    nss = NSSParams(a=0.040, b=-0.020, c=0.015, d=0.005, tau=1.5, theta=6.0)

    print("=== NSS examples ===")
    for T in [0.5, 1, 2, 5, 10]:
        y = zcy_nss(T, nss)
        P = P0T_from_nss(T, nss)
        f = fwd_nss(T, nss)
        print(f"T={T:>4}  zcy={y:.6f}  P0T={P:.6f}  fwd={f:.6f}")

    # --- Hull-White option on ZC using NSS curve
    hw = HullWhiteParams(a=0.10, sigma=0.01)
    K, S, T = 0.80, 2.0, 5.0
    call_hw = zc_option_hw_from_nss(K, S, T, nss, hw, option_type="call")
    print("\n=== HW + NSS ZC option example ===")
    print(f"Call on ZC: K={K}, expiry S={S}, maturity T={T} => price={call_hw:.6f}")

    # --- Vasicek ZC + option example
    vas = VasicekParams(kappa=0.30, mu=0.05, sigma=0.02, r0=0.04)
    P5 = P0T_vasicek(5.0, vas)
    call_v = zc_option_vasicek(K=0.80, S=2.0, T=5.0, vp=vas, option_type="call")
    print("\n=== Vasicek examples ===")
    print(f"P(0,5)={P5:.6f}, CallZC={call_v:.6f}")


=== NSS examples ===
T= 0.5  zcy=0.025197  P0T=0.987480  fwd=0.029635
T=   1  zcy=0.029023  P0T=0.971395  fwd=0.035571
T=   2  zcy=0.033954  P0T=0.934347  fwd=0.041194
T=   5  zcy=0.039238  P0T=0.821857  fwd=0.042881
T=  10  zcy=0.040721  P0T=0.665505  fwd=0.041676

=== HW + NSS ZC option example ===
Call on ZC: K=0.8, expiry S=2.0, maturity T=5.0 => price=0.074396

=== Vasicek examples ===
P(0,5)=0.801730, CallZC=0.066890


---
## 📝 Exemples de Questions de Quiz

### Question 1 : Modèle Nelson-Siegel-Svensson (NSS)

**Exemple de question typique :**
> "Étant donné les paramètres NSS suivants : a=0.035, b=-0.015, c=0.020, d=0.008, τ=2.0, θ=5.0
> 
> a) Calculez le taux zéro-coupon zcy(T) pour T = 1, 3, 5, 10 ans
> 
> b) Calculez le facteur d'escompte P(0,T) pour T = 5 ans
> 
> c) Calculez le taux forward instantané f(0,T) pour T = 3 ans"

**Comment utiliser les fonctions :**
- `zcy_nss(T, p)` → retourne le taux zéro-coupon (en décimal, ex: 0.035 = 3.5%)
- `P0T_from_nss(T, p)` → retourne le facteur d'escompte P(0,T)
- `fwd_nss(T, p)` → retourne le taux forward instantané

**Interprétation des résultats :**
- Si zcy(5) = 0.0372 → le taux annuel pour un placement de 5 ans est 3.72%
- Si P(0,5) = 0.8312 → 1€ dans 5 ans vaut 83.12 centimes aujourd'hui
- Si f(0,3) = 0.0410 → le taux forward instantané dans 3 ans est 4.10%

In [9]:
# ============================================
# EXEMPLE QUESTION 1 : NSS
# ============================================

# Définir les paramètres NSS donnés dans la question
nss_q1 = NSSParams(a=0.035, b=-0.015, c=0.020, d=0.008, tau=2.0, theta=5.0)

print("=" * 60)
print("QUESTION 1 : Modèle NSS")
print("=" * 60)

# a) Taux zéro-coupon pour différentes maturités
print("\na) Taux zéro-coupon zcy(T) :")
maturites = [1, 3, 5, 10]
for T in maturites:
    y = zcy_nss(T, nss_q1)
    print(f"   T = {T:2d} ans → zcy({T:2d}) = {y:.6f} = {y*100:.4f}%")

# b) Facteur d'escompte pour T = 5 ans
T_b = 5
P_b = P0T_from_nss(T_b, nss_q1)
print(f"\nb) Facteur d'escompte :")
print(f"   P(0,{T_b}) = {P_b:.6f}")
print(f"   → 1€ dans {T_b} ans vaut {P_b:.4f}€ aujourd'hui")

# c) Taux forward instantané pour T = 3 ans
T_c = 3
f_c = fwd_nss(T_c, nss_q1)
print(f"\nc) Taux forward instantané :")
print(f"   f(0,{T_c}) = {f_c:.6f} = {f_c*100:.4f}%")
print(f"   → Le taux 'instantané' prévu dans {T_c} ans est {f_c*100:.4f}%")

print("\n" + "=" * 60)

QUESTION 1 : Modèle NSS

a) Taux zéro-coupon zcy(T) :
   T =  1 ans → zcy( 1) = 0.027505 = 2.7505%
   T =  3 ans → zcy( 3) = 0.034752 = 3.4752%
   T =  5 ans → zcy( 5) = 0.037308 = 3.7308%
   T = 10 ans → zcy(10) = 0.038234 = 3.8234%

b) Facteur d'escompte :
   P(0,5) = 0.829825
   → 1€ dans 5 ans vaut 0.8298€ aujourd'hui

c) Taux forward instantané :
   f(0,3) = 0.040981 = 4.0981%
   → Le taux 'instantané' prévu dans 3 ans est 4.0981%



### Question 2 : Modèle de Vasicek

**Exemple de question typique :**
> "Sous le modèle de Vasicek avec κ=0.25, μ=0.06, σ=0.015, r₀=0.045 :
> 
> a) Calculez le prix d'une obligation zéro-coupon P(0,T) pour T = 5 ans
> 
> b) Calculez le prix d'une option call européenne sur cette obligation avec strike K=0.78, expirant dans S=2 ans"

**Comment utiliser les fonctions :**
- `P0T_vasicek(T, vp)` → prix de l'obligation zéro-coupon à maturité T
- `zc_option_vasicek(K, S, T, vp, option_type)` → prix de l'option (call ou put)

**Interprétation des résultats :**
- Si P(0,5) = 0.7928 → une obligation de valeur faciale 100€ dans 5 ans vaut 79.28€ aujourd'hui
- Si Call = 0.0234 → le prix de l'option call est 2.34€ (pour 100€ de nominal)
- Plus σ est grand, plus la volatilité des taux est élevée, plus l'option vaut cher

In [10]:
# ============================================
# EXEMPLE QUESTION 2 : Modèle de Vasicek
# ============================================

# Paramètres du modèle Vasicek
vas_q2 = VasicekParams(kappa=0.25, mu=0.06, sigma=0.015, r0=0.045)

print("=" * 60)
print("QUESTION 2 : Modèle de Vasicek")
print("=" * 60)
print(f"Paramètres : κ={vas_q2.kappa}, μ={vas_q2.mu}, σ={vas_q2.sigma}, r₀={vas_q2.r0}")

# a) Prix de l'obligation zéro-coupon
T_a = 5
P_a = P0T_vasicek(T_a, vas_q2)
print(f"\na) Prix de l'obligation zéro-coupon :")
print(f"   P(0,{T_a}) = {P_a:.6f}")
print(f"   → Pour 100€ de nominal : {P_a*100:.2f}€")

# b) Option call sur l'obligation
K_b = 0.78
S_b = 2
T_b = 5  # même maturité que l'obligation
call_b = zc_option_vasicek(K_b, S_b, T_b, vas_q2, option_type="call")
put_b = zc_option_vasicek(K_b, S_b, T_b, vas_q2, option_type="put")

print(f"\nb) Option sur l'obligation zéro-coupon P({S_b},{T_b}) :")
print(f"   Strike K = {K_b}")
print(f"   Expiration S = {S_b} ans")
print(f"   Maturité obligation T = {T_b} ans")
print(f"   Prix du Call = {call_b:.6f} ({call_b*100:.4f}€ pour 100€ nominal)")
print(f"   Prix du Put  = {put_b:.6f} ({put_b*100:.4f}€ pour 100€ nominal)")

# Vérification : parité call-put
parite = call_b - put_b
forward = P0T_vasicek(T_b, vas_q2) - K_b * P0T_vasicek(S_b, vas_q2)
print(f"\n   Vérification parité call-put :")
print(f"   Call - Put = {parite:.6f}")
print(f"   P(0,T) - K*P(0,S) = {forward:.6f}")
print(f"   Différence (devrait être ≈0) = {abs(parite - forward):.10f}")

print("\n" + "=" * 60)

QUESTION 2 : Modèle de Vasicek
Paramètres : κ=0.25, μ=0.06, σ=0.015, r₀=0.045

a) Prix de l'obligation zéro-coupon :
   P(0,5) = 0.774792
   → Pour 100€ de nominal : 77.48€

b) Option sur l'obligation zéro-coupon P(2,5) :
   Strike K = 0.78
   Expiration S = 2 ans
   Maturité obligation T = 5 ans
   Prix du Call = 0.066370 (6.6370€ pour 100€ nominal)
   Prix du Put  = 0.000051 (0.0051€ pour 100€ nominal)

   Vérification parité call-put :
   Call - Put = 0.066320
   P(0,T) - K*P(0,S) = 0.066320
   Différence (devrait être ≈0) = 0.0000000000



### Question 3 : Modèle CIR (Cox-Ingersoll-Ross)

**Exemple de question typique :**
> "Sous le modèle CIR avec κ=0.30, μ=0.05, σ=0.10, r₀=0.04 :
> 
> a) Calculez le prix d'une obligation zéro-coupon P(0,T) pour T = 3, 5, 10 ans
> 
> b) Comparez avec les prix sous Vasicek avec les mêmes paramètres"

**Comment utiliser les fonctions :**
- `P0T_cir(T, cp)` → prix de l'obligation zéro-coupon à maturité T sous CIR

**Interprétation des résultats :**
- Le modèle CIR garantit que les taux restent positifs (contrairement à Vasicek)
- La différence avec Vasicek vient du terme de volatilité σ√r (vs σ constant)
- γ = √(κ² + 2σ²) mesure la vitesse de retour vers la moyenne ajustée pour la volatilité

In [11]:
# ============================================
# EXEMPLE QUESTION 3 : Modèle CIR
# ============================================

# Paramètres du modèle CIR
cir_q3 = CIRParams(kappa=0.30, mu=0.05, sigma=0.10, r0=0.04)

print("=" * 60)
print("QUESTION 3 : Modèle CIR")
print("=" * 60)
print(f"Paramètres : κ={cir_q3.kappa}, μ={cir_q3.mu}, σ={cir_q3.sigma}, r₀={cir_q3.r0}")

# Calcul de gamma (paramètre important du modèle)
gamma = cir_gamma(cir_q3.kappa, cir_q3.sigma)
print(f"\nγ = √(κ² + 2σ²) = {gamma:.6f}")

# a) Prix des obligations zéro-coupon
print(f"\na) Prix des obligations zéro-coupon sous CIR :")
maturites_cir = [3, 5, 10]
prix_cir = {}
for T in maturites_cir:
    P = P0T_cir(T, cir_q3)
    prix_cir[T] = P
    print(f"   P(0,{T:2d}) = {P:.6f} (soit {P*100:.2f}€ pour 100€ nominal)")

# b) Comparaison avec Vasicek (mêmes paramètres)
vas_q3 = VasicekParams(kappa=cir_q3.kappa, mu=cir_q3.mu, sigma=cir_q3.sigma, r0=cir_q3.r0)

print(f"\nb) Comparaison CIR vs Vasicek :")
print(f"   {'Maturité':<10} {'CIR':<12} {'Vasicek':<12} {'Différence':<12} {'Diff %'}")
print(f"   {'-'*60}")
for T in maturites_cir:
    P_cir = prix_cir[T]
    P_vas = P0T_vasicek(T, vas_q3)
    diff = P_cir - P_vas
    diff_pct = (diff / P_vas) * 100
    print(f"   T={T:<8d} {P_cir:<12.6f} {P_vas:<12.6f} {diff:+12.6f} {diff_pct:+7.3f}%")

print(f"\n   Note : Les différences proviennent du fait que CIR utilise")
print(f"          une volatilité σ√r (garantit r≥0) vs σ constant (Vasicek)")

print("\n" + "=" * 60)

QUESTION 3 : Modèle CIR
Paramètres : κ=0.3, μ=0.05, σ=0.1, r₀=0.04

γ = √(κ² + 2σ²) = 0.331662

a) Prix des obligations zéro-coupon sous CIR :
   P(0, 3) = 0.878787 (soit 87.88€ pour 100€ nominal)
   P(0, 5) = 0.801875 (soit 80.19€ pour 100€ nominal)
   P(0,10) = 0.634136 (soit 63.41€ pour 100€ nominal)

b) Comparaison CIR vs Vasicek :
   Maturité   CIR          Vasicek      Différence   Diff %
   ------------------------------------------------------------
   T=3        0.878787     0.899376        -0.020589  -2.289%
   T=5        0.801875     0.864094        -0.062219  -7.201%
   T=10       0.634136     0.841694        -0.207558 -24.660%

   Note : Les différences proviennent du fait que CIR utilise
          une volatilité σ√r (garantit r≥0) vs σ constant (Vasicek)



### Question 4 : Modèle Hull-White 1 facteur avec courbe NSS

**Exemple de question typique :**
> "Utilisez la courbe NSS (a=0.04, b=-0.02, c=0.015, d=0.005, τ=1.5, θ=6.0) comme courbe initiale.
> Sous Hull-White avec a=0.12, σ=0.01 :
> 
> a) Calculez le prix d'une obligation zéro-coupon P(0,5)
> 
> b) Calculez le prix d'un call européen sur P(S,T) avec K=0.82, S=2 ans, T=5 ans"

**Comment utiliser les fonctions :**
- `P0T_hw_from_nss(T, nss)` → prix ZC à partir de la courbe NSS (courbe de marché)
- `zc_option_hw_from_nss(K, S, T, nss, hw, option_type)` → option sur ZC sous HW

**Interprétation des résultats :**
- Hull-White s'ajuste exactement à la courbe initiale (ici NSS)
- Le paramètre `a` contrôle la vitesse de retour à la moyenne
- Le paramètre `σ` contrôle la volatilité des taux courts
- L'option est plus chère si σ est élevé ou si S est loin

In [12]:
# ============================================
# EXEMPLE QUESTION 4 : Modèle Hull-White avec courbe NSS
# ============================================

# Courbe de marché initiale (NSS)
nss_q4 = NSSParams(a=0.04, b=-0.02, c=0.015, d=0.005, tau=1.5, theta=6.0)

# Paramètres Hull-White
hw_q4 = HullWhiteParams(a=0.12, sigma=0.01)

print("=" * 60)
print("QUESTION 4 : Modèle Hull-White 1F avec courbe NSS")
print("=" * 60)
print(f"Courbe NSS : a={nss_q4.a}, b={nss_q4.b}, c={nss_q4.c}, d={nss_q4.d}")
print(f"             τ={nss_q4.tau}, θ={nss_q4.theta}")
print(f"Paramètres HW : a={hw_q4.a}, σ={hw_q4.sigma}")

# a) Prix de l'obligation zéro-coupon (directement de la courbe NSS)
T_a = 5
P_a = P0T_hw_from_nss(T_a, nss_q4)
y_a = zcy_nss(T_a, nss_q4)

print(f"\na) Prix de l'obligation zéro-coupon :")
print(f"   P(0,{T_a}) = {P_a:.6f}")
print(f"   zcy({T_a}) = {y_a:.6f} = {y_a*100:.4f}%")
print(f"   Note : HW s'ajuste parfaitement à la courbe NSS initiale")

# b) Option call sur l'obligation
K_b = 0.82
S_b = 2
T_b = 5

call_b = zc_option_hw_from_nss(K_b, S_b, T_b, nss_q4, hw_q4, option_type="call")
put_b = zc_option_hw_from_nss(K_b, S_b, T_b, nss_q4, hw_q4, option_type="put")

print(f"\nb) Option européenne sur P({S_b},{T_b}) :")
print(f"   Strike K = {K_b}")
print(f"   Expiration S = {S_b} ans")
print(f"   Maturité obligation T = {T_b} ans")
print(f"   Prix du Call = {call_b:.6f} ({call_b*100:.4f}€ pour 100€)")
print(f"   Prix du Put  = {put_b:.6f} ({put_b*100:.4f}€ pour 100€)")

# Analyse de sensibilité à sigma
print(f"\nc) Sensibilité au paramètre de volatilité σ :")
sigmas = [0.005, 0.01, 0.015, 0.02]
print(f"   {'σ':<10} {'Call':<12} {'Put':<12}")
print(f"   {'-'*35}")
for sig in sigmas:
    hw_temp = HullWhiteParams(a=hw_q4.a, sigma=sig)
    c = zc_option_hw_from_nss(K_b, S_b, T_b, nss_q4, hw_temp, option_type="call")
    p = zc_option_hw_from_nss(K_b, S_b, T_b, nss_q4, hw_temp, option_type="put")
    print(f"   {sig:<10.4f} {c:<12.6f} {p:<12.6f}")

print(f"\n   → Plus σ est élevé, plus les options valent cher (plus d'incertitude)")

print("\n" + "=" * 60)

QUESTION 4 : Modèle Hull-White 1F avec courbe NSS
Courbe NSS : a=0.04, b=-0.02, c=0.015, d=0.005
             τ=1.5, θ=6.0
Paramètres HW : a=0.12, σ=0.01

a) Prix de l'obligation zéro-coupon :
   P(0,5) = 0.821857
   zcy(5) = 0.039238 = 3.9238%
   Note : HW s'ajuste parfaitement à la courbe NSS initiale

b) Option européenne sur P(2,5) :
   Strike K = 0.82
   Expiration S = 2 ans
   Maturité obligation T = 5 ans
   Prix du Call = 0.055812 (5.5812€ pour 100€)
   Prix du Put  = 0.000120 (0.0120€ pour 100€)

c) Sensibilité au paramètre de volatilité σ :
   σ          Call         Put         
   -----------------------------------
   0.0050     0.055693     0.000000    
   0.0100     0.055812     0.000120    
   0.0150     0.056869     0.001177    
   0.0200     0.059116     0.003423    

   → Plus σ est élevé, plus les options valent cher (plus d'incertitude)



### Question 5 : Pricing d'obligations à coupons

**Exemple de question typique :**
> "Une obligation paie des coupons de 5€ aux dates t=1, 2, 3 ans et rembourse 105€ à t=3 ans.
> Utilisez la courbe NSS pour calculer le prix de cette obligation aujourd'hui."

**Comment utiliser les fonctions :**
- `coupon_bond_price_from_curve(cashflows, times, P0T_func)` → prix de l'obligation

**Interprétation des résultats :**
- Le prix est la somme des flux actualisés : Σ CF_i × P(0,t_i)
- Si prix = 102.45€ pour 100€ de nominal + coupons, l'obligation cote au-dessus du pair
- Cela signifie que le taux de coupon (5%) est supérieur aux taux du marché

In [13]:
# ============================================
# EXEMPLE QUESTION 5 : Obligation à coupons
# ============================================

# Utilisons la courbe NSS
nss_q5 = NSSParams(a=0.04, b=-0.02, c=0.015, d=0.005, tau=1.5, theta=6.0)

print("=" * 60)
print("QUESTION 5 : Pricing d'obligation à coupons")
print("=" * 60)

# Obligation payant 5€ par an pendant 3 ans + 100€ de principal à la fin
coupon_annuel = 5
principal = 100
times = np.array([1.0, 2.0, 3.0])
cashflows = np.array([coupon_annuel, coupon_annuel, coupon_annuel + principal])

print(f"\nObligation à coupons :")
print(f"   Coupon annuel = {coupon_annuel}€")
print(f"   Principal = {principal}€")
print(f"   Dates de paiement : {times}")
print(f"   Flux de trésorerie : {cashflows}")

# Prix de l'obligation
prix_obligation = coupon_bond_price_from_curve(
    cashflows, 
    times, 
    lambda t: P0T_from_nss(t, nss_q5)
)

print(f"\n   Prix de l'obligation = {prix_obligation:.4f}€")

# Détail du calcul
print(f"\n   Détail du calcul :")
print(f"   {'Date':<8} {'Flux':<10} {'P(0,t)':<12} {'Valeur actuelle':<15}")
print(f"   {'-'*50}")
total_check = 0
for t, cf in zip(times, cashflows):
    P = P0T_from_nss(t, nss_q5)
    VA = cf * P
    total_check += VA
    print(f"   t={t:<6.1f} {cf:<10.2f} {P:<12.6f} {VA:<15.4f}")
print(f"   {'-'*50}")
print(f"   TOTAL = {total_check:.4f}€")

# Taux de rendement actuariel (YTM) implicite
taux_coupon = (coupon_annuel / principal) * 100
print(f"\n   Taux de coupon = {taux_coupon:.2f}%")
print(f"   Prix {'>' if prix_obligation > principal else '<'} {principal}€ → "
      f"YTM {'<' if prix_obligation > principal else '>'} taux de coupon")

print("\n" + "=" * 60)

QUESTION 5 : Pricing d'obligation à coupons

Obligation à coupons :
   Coupon annuel = 5€
   Principal = 100€
   Dates de paiement : [1. 2. 3.]
   Flux de trésorerie : [  5   5 105]

   Prix de l'obligation = 103.5790€

   Détail du calcul :
   Date     Flux       P(0,t)       Valeur actuelle
   --------------------------------------------------
   t=1.0    5.00       0.971395     4.8570         
   t=2.0    5.00       0.934347     4.6717         
   t=3.0    105.00     0.895717     94.0503        
   --------------------------------------------------
   TOTAL = 103.5790€

   Taux de coupon = 5.00%
   Prix > 100€ → YTM < taux de coupon



### Question 6 : Option sur obligation à coupons (Décomposition de Jamshidian)

**Exemple de question typique :**
> "Sous Hull-White (a=0.10, σ=0.01) avec courbe NSS, calculez le prix d'un call européen expirant dans S=1 an avec strike K=102€ sur une obligation qui paie :
> - 4€ à t=2 ans
> - 4€ à t=3 ans  
> - 104€ à t=4 ans"

**Comment utiliser les fonctions :**
- `coupon_bond_option_hw_from_nss_jamshidian(K, S, cashflows, times, nss, hw, option_type)`

**Interprétation des résultats :**
- La décomposition de Jamshidian transforme l'option sur obligation à coupons en un portefeuille d'options sur obligations zéro-coupon
- Trouve un taux critique r* tel que le prix de l'obligation = K à l'expiration
- Chaque flux est ensuite valorisé comme une option ZC avec un strike ajusté
- Plus complexe qu'une option ZC simple, mais prix exact pour modèles Gaussiens (Vasicek, HW)

In [14]:
# ============================================
# EXEMPLE QUESTION 6 : Option sur obligation à coupons (Jamshidian)
# ============================================

# Courbe NSS et paramètres Hull-White
nss_q6 = NSSParams(a=0.04, b=-0.02, c=0.015, d=0.005, tau=1.5, theta=6.0)
hw_q6 = HullWhiteParams(a=0.10, sigma=0.01)

print("=" * 60)
print("QUESTION 6 : Option sur obligation à coupons (Jamshidian)")
print("=" * 60)

# Obligation sous-jacente
times_q6 = np.array([2.0, 3.0, 4.0])
cashflows_q6 = np.array([4.0, 4.0, 104.0])

print(f"\nObligation sous-jacente :")
print(f"   Flux de trésorerie : {cashflows_q6}")
print(f"   Dates : {times_q6}")

# Prix actuel de l'obligation
prix_actuel = coupon_bond_price_from_curve(
    cashflows_q6, 
    times_q6, 
    lambda t: P0T_from_nss(t, nss_q6)
)
print(f"   Prix actuel de l'obligation = {prix_actuel:.4f}€")

# Option call avec expiration S=1 an et strike K=102
S_q6 = 1.0
K_q6 = 102.0

print(f"\nOption call :")
print(f"   Strike K = {K_q6}€")
print(f"   Expiration S = {S_q6} an")

try:
    call_q6 = coupon_bond_option_hw_from_nss_jamshidian(
        K_q6, S_q6, cashflows_q6, times_q6, nss_q6, hw_q6, option_type="call"
    )
    put_q6 = coupon_bond_option_hw_from_nss_jamshidian(
        K_q6, S_q6, cashflows_q6, times_q6, nss_q6, hw_q6, option_type="put"
    )
    
    print(f"   Prix du Call = {call_q6:.6f}€")
    print(f"   Prix du Put  = {put_q6:.6f}€")
    
    # Valeur intrinsèque approximative
    print(f"\n   Valeur intrinsèque approximative du call :")
    intrinsic_approx = max(prix_actuel - K_q6, 0)
    print(f"   max(Prix actuel - K, 0) ≈ {intrinsic_approx:.4f}€")
    print(f"   Note : Cette approx. ignore l'actualisation jusqu'à S")
    
    # Analyse de sensibilité au strike
    print(f"\n   Sensibilité au strike K :")
    strikes = [98, 100, 102, 104, 106]
    print(f"   {'Strike K':<12} {'Call':<12} {'Put':<12}")
    print(f"   {'-'*40}")
    for K in strikes:
        c = coupon_bond_option_hw_from_nss_jamshidian(
            K, S_q6, cashflows_q6, times_q6, nss_q6, hw_q6, option_type="call"
        )
        p = coupon_bond_option_hw_from_nss_jamshidian(
            K, S_q6, cashflows_q6, times_q6, nss_q6, hw_q6, option_type="put"
        )
        print(f"   {K:<12.2f} {c:<12.6f} {p:<12.6f}")
    
    print(f"\n   → Plus le strike est bas, plus le call vaut cher")
    print(f"   → Plus le strike est haut, plus le put vaut cher")
    
except Exception as e:
    print(f"   Erreur : {e}")

print("\n" + "=" * 60)

QUESTION 6 : Option sur obligation à coupons (Jamshidian)

Obligation sous-jacente :
   Flux de trésorerie : [  4.   4. 104.]
   Dates : [2. 3. 4.]
   Prix actuel de l'obligation = 96.5496€

Option call :
   Strike K = 102.0€
   Expiration S = 1.0 an
   Prix du Call = 0.163577€
   Prix du Put  = 2.696257€

   Valeur intrinsèque approximative du call :
   max(Prix actuel - K, 0) ≈ 0.0000€
   Note : Cette approx. ignore l'actualisation jusqu'à S

   Sensibilité au strike K :
   Strike K     Call         Put         
   ----------------------------------------
   98.00        1.741931     0.389033    
   100.00       0.654365     1.244256    
   102.00       0.163577     2.696257    
   104.00       0.025755     4.501224    
   106.00       0.002491     6.420749    

   → Plus le strike est bas, plus le call vaut cher
   → Plus le strike est haut, plus le put vaut cher



### Question 7 : Caps et Floors (décomposition en caplets/floorlets)

**Exemple de question typique :**
> "Un cap de notional 1M€, strike 3%, sur 3 périodes annuelles (t=1, 2, 3 ans).
> Utilisez Hull-White avec a=0.10, σ=0.01 et la courbe NSS."

**Comment utiliser les fonctions :**
- `cap_floor_from_zc_options(notional, K, schedule, nss, hw, kind)` → Prix du cap/floor

**Formules du cours (Équations 1.40 et 1.41) :**
- **CAP** : `Cap(M,K) = Σ M'·Put(P(t_{j-1}, t_j), K')`
- **FLOOR** : `Floor(M,K) = Σ M'·Call(P(t_{j-1}, t_j), K')`
- où `M' = M(1 + Kτ)` et `K' = 1/(1 + Kτ)`

**Interprétation des résultats :**
- Un **cap** protège contre la hausse des taux (= portefeuille de puts sur ZC)
- Un **floor** protège contre la baisse des taux (= portefeuille de calls sur ZC)
- Cap - Floor = Swap à taux fixe K (parité cap-floor)
- Plus le taux strike K est élevé, plus le cap est cher (et le floor moins cher)

In [20]:
# ============================================
# EXEMPLE QUESTION 7 : Caps et Floors
# ============================================

# Courbe NSS et paramètres Hull-White
nss_q7 = NSSParams(a=0.04, b=-0.02, c=0.015, d=0.005, tau=1.5, theta=6.0)
hw_q7 = HullWhiteParams(a=0.10, sigma=0.01)

print("=" * 60)
print("QUESTION 7 : Caps et Floors")
print("=" * 60)

print("""
✅ Fonction implémentée avec les formules du cours (Éq. 1.40 et 1.41) :

   CAP(M,K)   = Σ M'·Put(P(T_i, T_{i+1}), K')
   FLOOR(M,K) = Σ M'·Call(P(T_i, T_{i+1}), K')
   
   où M' = M(1 + Kτ) et K' = 1/(1 + Kτ)
""")

# Paramètres du cap/floor
notional = 1_000_000  # 1M€
strike_rate = 0.03    # 3%
# Note: Pour un cap, le taux de référence est observé en T_i et le payoff se fait en T_{i+1}
# Le premier caplet observe le taux en t=1 (pas en t=0) et paie en t=2
schedule = [
    (1.0, 2.0, 1.0),  # (T_i, T_{i+1}, delta) - Caplet 1: observe à t=1, paie à t=2
    (2.0, 3.0, 1.0),  # Caplet 2: observe à t=2, paie à t=3
    (3.0, 4.0, 1.0),  # Caplet 3: observe à t=3, paie à t=4
]

print(f"Paramètres :")
print(f"   Notional M = {notional:,.0f}€")
print(f"   Strike K = {strike_rate*100:.2f}%")
print(f"   Périodes : 3 caplets/floorlets annuels")

# Calcul du CAP
cap_price = cap_floor_from_zc_options(
    notional=notional,
    K=strike_rate,
    schedule=schedule,
    nss=nss_q7,
    hw=hw_q7,
    kind="cap"
)

# Calcul du FLOOR
floor_price = cap_floor_from_zc_options(
    notional=notional,
    K=strike_rate,
    schedule=schedule,
    nss=nss_q7,
    hw=hw_q7,
    kind="floor"
)

print(f"\na) Prix du Cap et du Floor :")
print(f"   Cap   = {cap_price:,.2f}€")
print(f"   Floor = {floor_price:,.2f}€")

# Détail par caplet/floorlet
print(f"\nb) Décomposition par caplet/floorlet :")
print(f"   {'Période':<15} {'M\'':<15} {'K\'':<12} {'Cap contrib.':<15} {'Floor contrib.'}")
print(f"   {'-'*75}")

total_cap = 0
total_floor = 0
for i, (Ti, Tip1, delta) in enumerate(schedule, 1):
    M_prime = notional * (1.0 + strike_rate * delta)
    K_prime = 1.0 / (1.0 + strike_rate * delta)
    
    cap_contrib = M_prime * zc_option_hw_from_nss(
        K_prime, Ti, Tip1, nss_q7, hw_q7, option_type="put"
    )
    floor_contrib = M_prime * zc_option_hw_from_nss(
        K_prime, Ti, Tip1, nss_q7, hw_q7, option_type="call"
    )
    
    total_cap += cap_contrib
    total_floor += floor_contrib
    
    print(f"   [{Ti:.1f}, {Tip1:.1f}]{'':>5} {M_prime:>14,.0f} {K_prime:>11.6f} {cap_contrib:>14,.2f} {floor_contrib:>15,.2f}")

print(f"   {'-'*75}")
print(f"   {'TOTAL':>28} {total_cap:>28,.2f} {total_floor:>15,.2f}")

# Parité Cap-Floor
print(f"\nc) Vérification de la parité Cap-Floor :")
diff = cap_price - floor_price
print(f"   Cap - Floor = {diff:,.2f}€")
print(f"   Cette différence correspond à la valeur d'un swap à taux fixe {strike_rate*100:.1f}%")

# Sensibilité au strike
print(f"\nd) Sensibilité au strike K :")
strikes_test = [0.02, 0.025, 0.03, 0.035, 0.04]
print(f"   {'Strike K':<12} {'Cap':<15} {'Floor':<15} {'Cap-Floor'}")
print(f"   {'-'*60}")

for K_test in strikes_test:
    c = cap_floor_from_zc_options(notional, K_test, schedule, nss_q7, hw_q7, kind="cap")
    f = cap_floor_from_zc_options(notional, K_test, schedule, nss_q7, hw_q7, kind="floor")
    print(f"   {K_test*100:<11.2f}% {c:>14,.2f} {f:>14,.2f} {c-f:>14,.2f}")

print(f"\n   → Strike ↑ : Cap ↑ (plus de protection), Floor ↓")

print("\n" + "=" * 60)

QUESTION 7 : Caps et Floors

✅ Fonction implémentée avec les formules du cours (Éq. 1.40 et 1.41) :

   CAP(M,K)   = Σ M'·Put(P(T_i, T_{i+1}), K')
   FLOOR(M,K) = Σ M'·Call(P(T_i, T_{i+1}), K')

   où M' = M(1 + Kτ) et K' = 1/(1 + Kτ)

Paramètres :
   Notional M = 1,000,000€
   Strike K = 3.00%
   Périodes : 3 caplets/floorlets annuels

a) Prix du Cap et du Floor :
   Cap   = 35,527.79€
   Floor = 2,748.50€

b) Décomposition par caplet/floorlet :
   Période         M'              K'           Cap contrib.    Floor contrib.
   ---------------------------------------------------------------------------
   [1.0, 2.0]           1,030,000    0.970874       9,707.38          689.79
   [2.0, 3.0]           1,030,000    0.970874      12,639.49          881.14
   [3.0, 4.0]           1,030,000    0.970874      13,180.91        1,177.58
   ---------------------------------------------------------------------------
                          TOTAL                    35,527.79        2,748.50

c) 

---
## ✅ Résumé : Toutes les fonctions sont prêtes pour le quiz !

### 📊 Fonctions implémentées :

1. **NSS (Nelson-Siegel-Svensson)** ✓
   - `zcy_nss()` - Taux zéro-coupon
   - `fwd_nss()` - Taux forward instantané
   - `P0T_from_nss()` - Facteur d'escompte

2. **Vasicek** ✓
   - `P0T_vasicek()` - Prix d'obligation ZC
   - `zc_option_vasicek()` - Options sur ZC

3. **CIR (Cox-Ingersoll-Ross)** ✓
   - `P0T_cir()` - Prix d'obligation ZC
   - `cir_gamma()`, `cir_A()`, `cir_B()` - Fonctions auxiliaires

4. **Hull-White 1F** ✓
   - `P0T_hw_from_nss()` - Prix ZC avec courbe NSS
   - `zc_option_hw_from_nss()` - Options sur ZC

5. **Obligations à coupons** ✓
   - `coupon_bond_price_from_curve()` - Pricing
   - `coupon_bond_option_hw_from_nss_jamshidian()` - Options (décomposition de Jamshidian)

6. **Caps & Floors** ✓
   - `cap_floor_from_zc_options()` - **MAINTENANT IMPLÉMENTÉ** avec les formules Éq. 1.40 et 1.41

### 🎯 Prêt pour le quiz !

Toutes les cellules d'exemples (Questions 1-7) sont exécutables et donnent des résultats concrets. Vous pouvez modifier les paramètres selon les questions de votre examen.